# fb testing

In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pickle
import dask as pd
import pandas as pd_old
import warnings
import utils
import heartpy as hp
from ECG_feature_extractor_1001 import *
# import time 
import time
from datetime import datetime
from biosppy.signals import ecg
from feature_extraction import SubjectData, compute_features, get_samples, combine_files

# To ignore all warnings:
warnings.filterwarnings("ignore", module="numpy")

In [481]:
WINDOW_IN_SECONDS = 60
stride = 1
label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0}
int_to_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
feat_names = None
loadPath = '../data/WESAD'
savePath = '../data/GN-WESAD'
subject_feature_path = '/subject_feats'
onedrive = '/mnt/d/Users/alkurdi/OneDrive - University of Illinois - Urbana/data/GN-WESAD'
n_samples = 10 
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
snrs = [ 0.0001, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6] #0.00001,
fb_model_list = ['DT', 'RF', 'LDA', 'KNN', 'AdaBoost', 'SVM']

if not os.path.exists(savePath):
    os.makedirs(savePath)
if not os.path.exists(savePath + subject_feature_path):
    os.makedirs(savePath + subject_feature_path)

In [3]:
def get_processing_status(snrs, subject_ids, onedrive, n_samples= 10):
    #bads are the ones that do not have the gaussian-modified data.  
    bads = []
    bad_snrs = []
    bad_subjects = []
    bad_ns = []
    completed_snrs = []
    for n_i in range(n_samples):
        for snr in snrs:
            for subject_id in subject_ids:
                #print(snr)
                
                #print(f'{onedrive}/n_{n_i}/snr_{snr}/S{subject_id}/{a}')
                try: 
                    a = os.listdir(f'{onedrive}/n_{n_i}/snr_{str(snr)}/S{subject_id}')
                    a[0]
                    completed_snrs.append(snr)
                except:
                    bads.append(f'n_{n_i}/snr_{snr}/S{subject_id}')
                    bad_snrs.append(snr)
                    bad_subjects.append(subject_id)
                    bad_ns.append(n_i)

    bad_snrs = sorted(set(bad_snrs))
    bad_subjects = sorted(set(bad_subjects))
    bad_ns = sorted(set(bad_ns))
    completed_snrs = sorted(set(completed_snrs))
    #printing after checking
    print(f'completed snrs :{completed_snrs}')
    print(f'incomplete snrs :{bad_snrs}')
get_processing_status(snrs, subject_ids, onedrive, n_samples= n_samples)

completed snrs :[0.0001, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6]
incomplete snrs :[]


In [4]:
dataset_list = []
for i in os.listdir(f'{loadPath}/subject_feats'):
    #print (i)
    if 'S' not in i[0]:
        dataset_list.append(i)
        
            
            

In [5]:
model_results = {}

for each_dataset in dataset_list:
    for snr in snrs:
        for model in fb_model_list:
            pass 

In [6]:
results_table = pd_old.DataFrame(columns=['SNR', 'Accuracy', 'F1-Score', 'dataset'])
results_table.loc[str('SVM')] = pd_old.Series({'SNR':1, 'Accuracy':5, 'F1 Score':2, 'dataset':'WESAD'})
results_table.loc[str('RF')] = pd_old.Series({'SNR':1, 'Accuracy':5, 'F1 Score':2, 'dataset':'WESAD'})
'''
fb_model_list = ['SVM', 'RF']
for model in fb_model_list:
    for i in range(len(snrs)):
        results_table.loc[str(model) + str(snrs[i])] = pd.Series({'SNR':snrs[i], 'Accuracy':svm_accuracy[i], 'F1 Score':2, 'dataset':'WESAD'})
'''
display(results_table)

,SNR,Accuracy,F1-Score,dataset
SVM,1,5,NaN,WESAD
RF,1,5,NaN,WESAD


In [7]:

start = time.time()

ecg_biosppy = ecg
ecg = None
fs_ecg = 700
fs_ppg = 64
subject_id=2 


In [8]:
subject = SubjectData(main_path=loadPath, subject_number=subject_id)
data_dict = subject.get_wrist_and_chest_data()

In [ ]:
ecg = data_dict['ECG'][40000:60000].flatten()
ppg = data_dict['BVP'][int(40000/700*64):int(60000/700*64)].flatten()
print('ecg len', len(ecg))
print('ppg len', len(ppg))


#testing for ECG

now = time.time()
#wd4, m4 = hp.process(ecg, fs_ecg)
#print('hp.process execution time is %5.2fs' % (time.time()-now))

now = time.time()
#pack2, ecgout2, time_dict2 = freq_ratio_hybrid(ecg, fs_ecg, RR1, method='welch', factor = 1)
#print('freq_ratio_hybrid execution time is %5.2fs' % (time.time()-now))



In [ ]:
now = time.time()
BS_signal_analysis3, pack3, ecg_out3 = analyze_ecg(ecg, fs_ecg)
print('analyze_ecg execution time is ', now-time.time())


In [ ]:
ecg = data_dict['ECG'][10000:80000].flatten()


In [140]:
#testing for PPG

now = datetime.now()
pack, ppg, RR, time_dict = freq_ratio( ppg, fs_ppg, method='welch', factor = 1)
print('freq_ratio execution time is ', now-time.time())

now = datetime.now()
pack, ppg, time_dict = freq_ratio_hybrid(ppg, fs_ppg, method='welch', factor = 1)
print('freq_ratio_hybrid execution time is ', now-time.time())

now = datetime.now()
BS_signal_analysis, pack, ppg_out = analyze_ecg(ppg, fs_ppg)
print('analyze_ecg execution time is ', now-time.time())

now = datetime.now()
wd, m = hp.process(ppg, fs_ppg)
print('hp.process execution time is ', now-time.time())

now = datetime.now()
pack, ppg, RR = freq_ratio_fast( ppg, fs_ppg, method='welch', factor = 1)
print('freq_ratio_fast execution time is ', now-time.time())

SyntaxError: invalid syntax (2154369309.py, line 4)

In [7]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
import logging


def combine_noiZ_files(subject_ids):
    today = datetime.now().strftime('%Y-%m-%d')
    logging.basicConfig(level=logging.DEBUG, filename=today+'-combine.log', filemode='w', force=True)
    logging.info('Started')
    print('total number of combines: ', len(snrs)*n_samples)
    df_list = []
    i = 0
    for snr in snrs:
        for n_i in range(n_samples):
            for s in subject_ids:
                df = pd_old.read_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/S{s}_feats.csv', index_col=0)
                df['subject'] = s
                df_list.append(df)
            df = pd_old.concat(df_list)
            df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
            df.drop(['0', '1', '2'], axis=1, inplace=True)
            df.reset_index(drop=True, inplace=True)
            now = datetime.now().strftime('%Y-%m-%d')
            df.to_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv')
            i+=1
            logging.info(f'Saved file to: {savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{today}_feats_filt.csv  {i}/{len(snrs)*n_samples}')
            print('Saved file to: ',f'.../n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv   {i}/{len(snrs)*n_samples}')
            counts = df['label'].value_counts()
            logging.info('Number of samples per class:')
            logging.info('baseline: {0[1]}; stress: {1[1]}; amusement: {2[1]} '.format(*list(zip(counts.index, counts.values))))
    logging.info('all done!')

In [8]:
combine_noiZ_files(subject_ids)
# took 19m 30s to run

total number of combines:  110
Saved file to:  .../n_0/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_1/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_2/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_3/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_4/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_5/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_6/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_7/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_8/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_9/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_0/snr_0.001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_1/snr_0.001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_2/snr_0.001/subject_feats/2023-11-12_feats_filt.c

In [233]:
import pandas as pd
import os
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import logging
now = datetime.now().strftime('%Y-%m-%d')

def process_subject_file(snr, n_i, s):
    file_path = f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/S{s}_feats.csv'
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0)
        df['subject'] = s
        return df
    return None

def combine_noiZ_files(subjects):
    now = datetime.now().strftime('%Y-%m-%d')
    logging.basicConfig(level=logging.INFO, filename=now+'-combine.log', filemode='w', force=True)

    for snr in snrs:
        for n_i in range(n_samples):
            # Parallelize file reading
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(process_subject_file, snr, n_i, s) for s in subjects]
                df_list = [future.result() for future in futures if future.result() is not None]

            if df_list:
                df = pd.concat(df_list)
                df['label'] = df[['0', '1', '2']].idxmax(axis=1)
                df.drop(['0', '1', '2'], axis=1, inplace=True)
                df.reset_index(drop=True, inplace=True)
                
                df.to_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv')
                logging.info('-' * 20)
                logging.info(f'Saved file to: {savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv')
                
                counts = df['label'].value_counts()
                logging.info('Number of samples per class:')
                logging.info('baseline: {0[1]}; stress: {1[1]}; amusement: {2[1]} '.format(*list(zip(counts.index, counts.values))))
    logging.info('all done!')

In [223]:
n_i = 5
s = 2
snr = 0.6
df = pd_old.read_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/S{subject_id}_feats.csv', index_col=0)
df['subject'] = s
#df_list.append(df)
#f = pd_old.concat(df_list)
df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
df.drop(['0', '1', '2'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

counts = df['label'].value_counts()
print('Number of samples per class:')
print('baseline: {0[1]}; stress: {1[1]}; amusement: {2[1]} '.format(*list(zip(counts.index, counts.values))))

Number of samples per class:
baseline: 119; stress: 61; amusement: 34 


In [234]:
combine_noiZ_files(subject_ids)
#this paralellized took 34seconds

In [33]:
logging.basicConfig(level=logging.INFO, filemode='w', force=True)
logging.info('GN-WESAD models ran and results generated and saved in: ')

INFO:root:GN-WESAD models ran and results generated and saved in: 


In [134]:
def read_n_reduce(path):
    #print(os.listdir(path))
    df = pd_old.read_csv(path, index_col=0)
    print('len of df ',len(df))
    pd_old.set_option('display.max_columns', None) 
    #We want to drop columns in df that are not in RADWear to match modalities. 
    # drop _c columns
    columns_list = df.columns.tolist()
    drop_list = []
    #df.drop(columns=['Resp_C'])
    for column in columns_list:
        if 'EMG' in column or 'EDA_C' in column or 'Temp_C' in column or 'TEMP_C' in column or 'SCR_C' in column or 'SCL_C' in column:
            drop_list.append(column)

    reduced_df = df.drop(columns=drop_list)
    df = reduced_df
    print('len of reduced df ',len(df))
    return df 
def gn_wesad_path(n_i, snr):
    loadPath = '../data/GN-WESAD'
    return f'{loadPath}/n_{n_i}/snr_{snr}{subject_feature_path}/{gn_wesad_day}_feats2.csv'
gn_wesad_day = '2023-11-13'
matrix = np.zeros((len(snrs), n_samples))

for n_i in range(n_samples):
    for i, snr in enumerate(snrs):
        print(f'for {n_i} and {snr} number {i}: ')
        file_path = gn_wesad_path(n_i, snr)
        df = read_n_reduce(file_path)
        matrix[i][n_i] = len(df)


for 0 and 0.0001 number 0: 
len of df  3355
len of reduced df  3355
for 0 and 0.001 number 1: 
len of df  3355
len of reduced df  3355
for 0 and 0.01 number 2: 
len of df  3355
len of reduced df  3355
for 0 and 0.05 number 3: 
len of df  3355
len of reduced df  3355
for 0 and 0.1 number 4: 
len of df  3355
len of reduced df  3355
for 0 and 0.15 number 5: 
len of df  3355
len of reduced df  3355
for 0 and 0.2 number 6: 
len of df  3355
len of reduced df  3355
for 0 and 0.3 number 7: 
len of df  3355
len of reduced df  3355
for 0 and 0.4 number 8: 
len of df  3355
len of reduced df  3355
for 0 and 0.5 number 9: 
len of df  3355
len of reduced df  3355
for 0 and 0.6 number 10: 
len of df  3355
len of reduced df  3355
for 1 and 0.0001 number 0: 
len of df  3355
len of reduced df  3355
for 1 and 0.001 number 1: 
len of df  3355
len of reduced df  3355
for 1 and 0.01 number 2: 
len of df  3355
len of reduced df  3355
for 1 and 0.05 number 3: 
len of df  3355
len of reduced df  3355
for 1 and

In [106]:
'data/GN-WESAD/n_0/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv'
'data/GN-WESAD/n_0/snr_0.001/subject_feats/2023-11-12_feats_filt.csv'
loadPath = '../data/GN-WESAD'
#display(os.listdir(f'{loadPath}/n_{n_i}/snr_{snr}{subject_feature_path}'))
snr0001 = pd_old.read_csv('../data/GN-WESAD/n_0/snr_0.0001/subject_feats/2023-11-13_feats2.csv', index_col=0)
snr001 = pd_old.read_csv('../data/GN-WESAD/n_0/snr_0.001/subject_feats/2023-11-13_feats2.csv', index_col=0)

In [141]:
import pandas as pd
df = pd.DataFrame({
    'SNR': [0.6]*10,
    'Accuracy': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'F1 Score': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'dataset': ['GN-WESAD']*10,
    'n_i': list(range(10)),
    'n': [10]*10,
    'noise gen function': ['Gaussian Noise']*10,
    'Precision': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'Recall': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'Model': ['DT']*10
})

# Example Series
series = pd.Series({
    'SNR': 0.600000,
    'Accuracy': 0.945747,
    'F1 Score': 0.945747,
    'n_i': 4.500000,
    'n': 10.000000,
    'Precision': 0.945747,
    'Recall': 0.945747
}, name='SNR 0.6 Model DT mean')

# Convert the series to a DataFrame
series_df = series.to_frame().T

# Append the new DataFrame to the existing one
df_combined = pd.concat([df, series_df], ignore_index=True)

display(df_combined)

,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model
0,0.6,0.940639,0.940639,GN-WESAD,0.0,10.0,Gaussian Noise,0.940639,0.940639,DT
1,0.6,0.968134,0.968134,GN-WESAD,1.0,10.0,Gaussian Noise,0.968134,0.968134,DT
2,0.6,0.964939,0.964939,GN-WESAD,2.0,10.0,Gaussian Noise,0.964939,0.964939,DT
3,0.6,0.940909,0.940909,GN-WESAD,3.0,10.0,Gaussian Noise,0.940909,0.940909,DT
4,0.6,0.937785,0.937785,GN-WESAD,4.0,10.0,Gaussian Noise,0.937785,0.937785,DT
5,0.6,0.923780,0.923780,GN-WESAD,5.0,10.0,Gaussian Noise,0.923780,0.923780,DT
6,0.6,0.942337,0.942337,GN-WESAD,6.0,10.0,Gaussian Noise,0.942337,0.942337,DT
7,0.6,0.960486,0.960486,GN-WESAD,7.0,10.0,Gaussian Noise,0.960486,0.960486,DT
8,0.6,0.952816,0.952816,GN-WESAD,8.0,10.0,Gaussian Noise,0.952816,0.952816,DT
9,0.6,0.925645,0.925645,GN-WESAD,9.0,10.0,Gaussian Noise,0.925645,0.925645,DT


In [157]:
df = pd.DataFrame({
    'SNR': [0.6]*10,
    'Accuracy': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'F1 Score': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'dataset': ['GN-WESAD']*10,
    'n_i': list(range(10)),
    'n': [10]*10,
    'noise gen function': ['Gaussian Noise']*10,
    'Precision': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'Recall': [0.940639, 0.968134, 0.964939, 0.940909, 0.937785, 0.923780, 0.942337, 0.960486, 0.952816, 0.925645],
    'Model': ['DT']*10
})
series = pd.Series({
    'SNR': 0.600000,
    'Accuracy': 0.945747,
    'F1 Score': 0.945747,
    'n_i': 4.500000,
    'n': 10.000000,
    'Precision': 0.945747,
    'Recall': 0.945747
}, name='SNR 0.6 Model DT mean')

print(set(df.columns))
print(set(series.index))
missing_columns = set(df.columns) - set(series.index)
print(missing_columns)
for col in missing_columns:
    # You can assign a specific value or use a value from the DataFrame
    # Here, I'm using the first row's value as an example
    #print(col)
    #print(df[col].iloc[0])
    
    series[col] = df[col].iloc[0]
    #print(series[col])
    
#print(series)

series_df2 = series.to_frame().T
display(series_df2)
df_combined2 = pd.concat([df, series_df2])
display(df_combined2)


{'n', 'Model', 'noise gen function', 'n_i', 'Accuracy', 'F1 Score', 'SNR', 'Precision', 'Recall', 'dataset'}
{'n', 'n_i', 'Accuracy', 'F1 Score', 'SNR', 'Precision', 'Recall'}
{'dataset', 'Model', 'noise gen function'}


,SNR,Accuracy,F1 Score,n_i,n,Precision,Recall,dataset,Model,noise gen function
SNR 0.6 Model DT mean,0.6,0.945747,0.945747,4.5,10.0,0.945747,0.945747,GN-WESAD,DT,Gaussian Noise


,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model
0,0.6,0.940639,0.940639,GN-WESAD,0,10,Gaussian Noise,0.940639,0.940639,DT
1,0.6,0.968134,0.968134,GN-WESAD,1,10,Gaussian Noise,0.968134,0.968134,DT
2,0.6,0.964939,0.964939,GN-WESAD,2,10,Gaussian Noise,0.964939,0.964939,DT
3,0.6,0.940909,0.940909,GN-WESAD,3,10,Gaussian Noise,0.940909,0.940909,DT
4,0.6,0.937785,0.937785,GN-WESAD,4,10,Gaussian Noise,0.937785,0.937785,DT
5,0.6,0.92378,0.92378,GN-WESAD,5,10,Gaussian Noise,0.92378,0.92378,DT
6,0.6,0.942337,0.942337,GN-WESAD,6,10,Gaussian Noise,0.942337,0.942337,DT
7,0.6,0.960486,0.960486,GN-WESAD,7,10,Gaussian Noise,0.960486,0.960486,DT
8,0.6,0.952816,0.952816,GN-WESAD,8,10,Gaussian Noise,0.952816,0.952816,DT
9,0.6,0.925645,0.925645,GN-WESAD,9,10,Gaussian Noise,0.925645,0.925645,DT


In [464]:
#tabulate and plot

with open('../data/GN-WESAD/cm_cr_dict.pickle', 'rb') as handle:
    GN_cm_cr_dict = pickle.load(handle)
with open('../data/WESAD/cm_cr_dict.pickle', 'rb') as handle:
    WESAD_cm_cr_dict = pickle.load(handle)
wesad_acc = pd.read_csv('../data/WESAD/wesad_models_results-win60stride1_wcm_wcr.csv', index_col=0)
display(wesad_acc)
#combined_results = pd.concat([WESAD_model_results, GN_model_results])


,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model
0,NaN,0.991058,0.991058,WESAD,1,1,Gaussian Noise,0.991058,0.991058,DT
1,NaN,0.916542,0.916542,WESAD,1,1,Gaussian Noise,0.916542,0.916542,RF
2,NaN,0.929955,0.929955,WESAD,1,1,Gaussian Noise,0.929955,0.929955,LDA
3,NaN,0.943368,0.943368,WESAD,1,1,Gaussian Noise,0.943368,0.943368,KNN
4,NaN,0.751118,0.751118,WESAD,1,1,Gaussian Noise,0.751118,0.751118,AdaBoost
5,NaN,0.950820,0.950820,WESAD,1,1,Gaussian Noise,0.950820,0.950820,SVM


In [485]:
fb_model_list

['DT', 'RF', 'LDA', 'KNN', 'AdaBoost', 'SVM']

In [507]:
with open('../data/GN-WESAD/cm_cr_dict.pickle', 'rb') as handle:
    GN_cm_cr_dict = pickle.load(handle)
tgt_file = '../data/GN-WESAD/GN_wesad_models_results_wbinaryf1.csv'
gn_wesad_acc = pd.read_csv(tgt_file, index_col=0)
#gn_wesad_acc['Binary F1'] = None
print(gn_wesad_acc.shape)

(660, 11)


In [508]:
gn_wesad_acc 

,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model,Binary F1
0,0.0001,0.731411,0.731411,GN-WESAD,0,10,Gaussian Noise,0.731411,0.731411,DT,0.794409
1,0.0001,0.567527,0.567527,GN-WESAD,0,10,Gaussian Noise,0.567527,0.567527,RF,0.723676
2,0.0001,0.562974,0.562974,GN-WESAD,0,10,Gaussian Noise,0.562974,0.562974,LDA,0.710756
3,0.0001,0.761760,0.761760,GN-WESAD,0,10,Gaussian Noise,0.761760,0.761760,KNN,0.837134
4,0.0001,0.587253,0.587253,GN-WESAD,0,10,Gaussian Noise,0.587253,0.587253,AdaBoost,0.715422
...,...,...,...,...,...,...,...,...,...,...,...
655,0.6000,0.825493,0.825493,GN-WESAD,9,10,Gaussian Noise,0.825493,0.825493,RF,0.866604
656,0.6000,0.839150,0.839150,GN-WESAD,9,10,Gaussian Noise,0.839150,0.839150,LDA,0.865120
657,0.6000,0.811836,0.811836,GN-WESAD,9,10,Gaussian Noise,0.811836,0.811836,KNN,0.857772
658,0.6000,0.676783,0.676783,GN-WESAD,9,10,Gaussian Noise,0.676783,0.676783,AdaBoost,0.723664


In [489]:
with open('../data/GN-WESAD/cm_cr_dict.pickle', 'rb') as handle:
    GN_cm_cr_dict = pickle.load(handle)
tgt_file = '../data/GN-WESAD/GN_wesad_models_results_wcm_wcr.csv'
gn_wesad_acc = pd.read_csv(tgt_file, index_col=0)
gn_wesad_acc['Binary F1'] = None

for i, classification_report in enumerate(GN_cm_cr_dict['cr']):
    cr = classification_report['Classification Report']
    print(classification_report['id'][2], fb_model_list[i%6])
    #print(cr)
    #print(i//5)
    binary_f1_score = calculate_binary_metrics(cr)
    #print(classification_report['id'])
    if binary_f1_score is not None:
        #print("Binary F1 Score for ", fb_model_list[i], " :\t\t", binary_f1_score)
        insta_acc = wesad_acc['Accuracy'][i%6]
        insta_f1 = wesad_acc['F1 Score'][i%6]
        secret_df = pd.concat([secret_df, pd.Series({'Model': fb_model_list[i%6], 'Binary F1 Score': binary_f1_score,
                                                    'SNR': classification_report['id'][0], 'n':classification_report['id'][1],
                                                    'Model': classification_report['id'][2]}).to_frame().T], ignore_index=True)
        #print(f'my calc acc {insta_acc}, calc f1 {insta_f1}')
        #display(wesad_acc_test[wesad_acc_test['Model'] == fb_model_list[i]])#['binary f1'] = binary_f1_score
        #wesad_acc_test[wesad_acc_test['Model'] == fb_model_list[i]]['binary f1'] = binary_f1_score
        #wesad_acc_test[wesad_acc_test['Model'] == fb_model_list[i]]['binary acc'] = insta_acc
        #print(fb_model_list[i])
        gn_wesad_acc.loc[gn_wesad_acc['Model'] == fb_model_list[i%6], 'Binary f1'] = binary_f1_score
        #print(gn_wesad_acc.loc[gn_wesad_acc['Model'] == fb_model_list[i], 'binary f1']) 

    else:
        print("Could not extract metrics from the report.")

DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN
AdaBoost AdaBoost
SVM SVM
DT DT
RF RF
LDA LDA
KNN KNN


In [454]:
print(len(GN_cm_cr_dict['cr']))
print(len(WESAD_cm_cr_dict['cr']))

660
6


In [425]:
wesad_acc_test = wesad_acc.copy()
display(wesad_acc_test)
#wesad_acc_test.loc['Model'] = 'DT'
model = 'DT'
#display (wesad_acc_test[wesad_acc_test['Model'] == model]['binary f1'] = 4)#[wesad_acc_test['SNR'] == np.nan] )

,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model
0,NaN,0.988077,0.988077,GN-WESAD,1,10,Gaussian Noise,0.988077,0.988077,DT
1,NaN,0.916542,0.916542,GN-WESAD,1,10,Gaussian Noise,0.916542,0.916542,RF
2,NaN,0.929955,0.929955,GN-WESAD,1,10,Gaussian Noise,0.929955,0.929955,LDA
3,NaN,0.943368,0.943368,GN-WESAD,1,10,Gaussian Noise,0.943368,0.943368,KNN
4,NaN,0.751118,0.751118,GN-WESAD,1,10,Gaussian Noise,0.751118,0.751118,AdaBoost
5,NaN,0.950820,0.950820,GN-WESAD,1,10,Gaussian Noise,0.950820,0.950820,SVM


In [415]:
for row in wesad_acc_test.iterrows():
    #if row[1]['Model'] == model:
    #    row[1]['binary f1'] = 4
    print(row)
    break

(0, SNR                              NaN
Accuracy                    0.988077
F1 Score                    0.988077
dataset                     GN-WESAD
n_i                                1
n                                 10
noise gen function    Gaussian Noise
Precision                   0.988077
Recall                      0.988077
Model                             DT
Name: 0, dtype: object)


In [427]:
wesad_acc_test['binary f1'] = None 
wesad_acc_test['binary acc'] = None
#display(wesad_acc_test.drop(columns=[['binary f1', 'wow']]))
display(wesad_acc_test)


,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model,binary f1,binary acc
0,NaN,0.988077,0.988077,GN-WESAD,1,10,Gaussian Noise,0.988077,0.988077,DT,None,None
1,NaN,0.916542,0.916542,GN-WESAD,1,10,Gaussian Noise,0.916542,0.916542,RF,None,None
2,NaN,0.929955,0.929955,GN-WESAD,1,10,Gaussian Noise,0.929955,0.929955,LDA,None,None
3,NaN,0.943368,0.943368,GN-WESAD,1,10,Gaussian Noise,0.943368,0.943368,KNN,None,None
4,NaN,0.751118,0.751118,GN-WESAD,1,10,Gaussian Noise,0.751118,0.751118,AdaBoost,None,None
5,NaN,0.950820,0.950820,GN-WESAD,1,10,Gaussian Noise,0.950820,0.950820,SVM,None,None


In [511]:
for result in WESAD_cm_cr_dict['cr']:
    #print('result', result)
    if True:#'.' in result['id']:
        #print(result['id'])
        print(result['Classification Report'])
        #print('------------------')

              precision    recall  f1-score   support

           0      0.962     0.981     0.971       104
           1      0.995     0.990     0.992       389
           2      1.000     1.000     1.000       178

    accuracy                          0.991       671
   macro avg      0.986     0.990     0.988       671
weighted avg      0.991     0.991     0.991       671

              precision    recall  f1-score   support

           0      1.000     0.481     0.649       104
           1      0.880     0.997     0.935       389
           2      0.983     0.994     0.989       178

    accuracy                          0.917       671
   macro avg      0.954     0.824     0.858       671
weighted avg      0.926     0.917     0.905       671

              precision    recall  f1-score   support

           0      0.764     0.808     0.785       104
           1      0.948     0.936     0.942       389
           2      0.994     0.989     0.992       178

    accuracy        

In [447]:
import re
wesad_acc_test = wesad_acc.copy()
wesad_acc_test['Binary F1'] = None
def extract_metrics(report):
    # Regular expression to find numeric values
    regex = r"\s+1\s+([\d\.]+)\s+([\d\.]+)\s+([\d\.]+)"

    # Search for the pattern
    match = re.search(regex, report)
    #print(match)
    if match:
        precision = float(match.group(1))
        recall = float(match.group(2))
        f1_score = float(match.group(3))
        return precision, recall, f1_score
    else:
        return None, None, None
    
def calculate_binary_metrics(report):
    precision, recall, f1_score = extract_metrics(report)

    if precision is not None and recall is not None:
        # Calculate binary F1 score for class 1
        binary_f1_score = 2 * (precision * recall) / (precision + recall)
        return binary_f1_score
    else:
        return None

#classification_report = 
# Extract and calculate binary F1 score
fb_model_counter = fb_model_list


secret_df = pd.DataFrame()

for i, classification_report in enumerate(WESAD_cm_cr_dict['cr']):
    cr = classification_report['Classification Report']
    binary_f1_score = calculate_binary_metrics(cr)
    #print(classification_report['id'])
    if binary_f1_score is not None:
        print("Binary F1 Score for ", fb_model_list[i], " :\t\t", binary_f1_score)
        insta_acc = wesad_acc['Accuracy'][i]
        insta_f1 = wesad_acc['F1 Score'][i]
        secret_df = pd.concat([secret_df, pd.Series({'Model': fb_model_list[i], 'Binary F1 Score': binary_f1_score,
                                                     'SNR': classification_report['id'][0], 'n':classification_report['id'][1],
                                                     'Model': classification_report['id'][2]}).to_frame().T], ignore_index=True)
        #print(f'my calc acc {insta_acc}, calc f1 {insta_f1}')
        #display(wesad_acc_test[wesad_acc_test['Model'] == fb_model_list[i]])#['binary f1'] = binary_f1_score
        #wesad_acc_test[wesad_acc_test['Model'] == fb_model_list[i]]['binary f1'] = binary_f1_score
        #wesad_acc_test[wesad_acc_test['Model'] == fb_model_list[i]]['binary acc'] = insta_acc
        #print(fb_model_list[i])
        wesad_acc_test.loc[wesad_acc_test['Model'] == fb_model_list[i], 'binary f1'] = binary_f1_score
        print(wesad_acc_test.loc[wesad_acc_test['Model'] == fb_model_list[i], 'binary f1']) 

    else:
        print("Could not extract metrics from the report.")
display(secret_df)
display(wesad_acc_test)


Binary F1 Score for  DT  :		 0.9894936836786254
0    0.989494
Name: binary f1, dtype: object
Binary F1 Score for  RF  :		 0.9348534896110815
1    0.934853
Name: binary f1, dtype: object
Binary F1 Score for  SVM  :		 0.9419617834394906
5    0.941962
Name: binary f1, dtype: object
Binary F1 Score for  LDA  :		 0.9579624217118997
2    0.957962
Name: binary f1, dtype: object
Binary F1 Score for  KNN  :		 0.802608910891089
3    0.802609
Name: binary f1, dtype: object
Binary F1 Score for  AdaBoost  :		 0.9574245430809398
4    0.957425
Name: binary f1, dtype: object


,Model,Binary F1 Score,SNR,n
0,DT,0.989494,N/A,1
1,RF,0.934853,N/A,1
2,LDA,0.941962,N/A,1
3,KNN,0.957962,N/A,1
4,AdaBoost,0.802609,N/A,1
5,SVM,0.957425,N/A,1


,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model,binary f1
0,NaN,0.988077,0.988077,GN-WESAD,1,10,Gaussian Noise,0.988077,0.988077,DT,0.989494
1,NaN,0.916542,0.916542,GN-WESAD,1,10,Gaussian Noise,0.916542,0.916542,RF,0.934853
2,NaN,0.929955,0.929955,GN-WESAD,1,10,Gaussian Noise,0.929955,0.929955,LDA,0.957962
3,NaN,0.943368,0.943368,GN-WESAD,1,10,Gaussian Noise,0.943368,0.943368,KNN,0.802609
4,NaN,0.751118,0.751118,GN-WESAD,1,10,Gaussian Noise,0.751118,0.751118,AdaBoost,0.957425
5,NaN,0.950820,0.950820,GN-WESAD,1,10,Gaussian Noise,0.950820,0.950820,SVM,0.941962


In [383]:
pd.concat([wesad_acc, secret_df], ignore_index=True)

,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model,Binary F1 Score
0,NaN,0.991058,0.991058,GN-WESAD,1.0,10,Gaussian Noise,0.991058,0.991058,DT,NaN
1,NaN,0.916542,0.916542,GN-WESAD,1.0,10,Gaussian Noise,0.916542,0.916542,RF,NaN
2,NaN,0.929955,0.929955,GN-WESAD,1.0,10,Gaussian Noise,0.929955,0.929955,LDA,NaN
3,NaN,0.943368,0.943368,GN-WESAD,1.0,10,Gaussian Noise,0.943368,0.943368,KNN,NaN
4,NaN,0.751118,0.751118,GN-WESAD,1.0,10,Gaussian Noise,0.751118,0.751118,AdaBoost,NaN
5,NaN,0.950820,0.950820,GN-WESAD,1.0,10,Gaussian Noise,0.950820,0.950820,SVM,NaN
6,N/A,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,DT,0.992
7,N/A,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,RF,0.934853
8,N/A,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,LDA,0.941962
9,N/A,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,KNN,0.957962


In [357]:
fafi = pd.DataFrame(columns=['SNR', 'Accuracy', 'F1 Score', 'dataset', 'n_i', 'n',
       'noise gen function', 'Precision', 'Recall', 'Model'])
pd.Series(classification_report['id']).to_frame().T
wow = (pd.Series(classification_report['id']).to_frame().T.rename(columns={0:'SNR', 1:'n', 2:'model'}))
fafi = pd.concat([fafi,wow ])
display(fafi)

,SNR,Accuracy,F1 Score,dataset,n_i,n,noise gen function,Precision,Recall,Model,model
0,N/A,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,SVM


In [364]:
classification_report['id'][2]

'DT'

In [310]:
len(WESAD_cm_cr_dict['cr'])
fb_model_list

6

In [319]:
wesad_acc['Accuracy'][i] 
wesad_acc['F1 Score'][i]

0.9508196721311476

In [181]:
def read_n_reduce(path):
    df = pd.read_csv(path, index_col=0)
    pd.set_option('display.max_columns', None) 
    #We want to drop columns in df that are not in RADWear to match modalities. 
    # drop _c columns
    columns_list = df.columns.tolist()
    drop_list = []
    #df.drop(columns=['Resp_C'])
    for column in columns_list:
        if 'EMG' in column or 'EDA_C' in column or 'Temp_C' in column or 'TEMP_C' in column or 'SCR_C' in column or 'SCL_C' in column:
            drop_list.append(column)

    reduced_df = df.drop(columns=drop_list)
    df = reduced_df
    return df 
print(os.listdir('../data/WESAD'))
df = read_n_reduce('../data/WESAD//subject_feats/oct5_feats4.csv')

['.ipynb_checkpoints', 'cm_cr_dict.pickle', 'desktop.ini', 'E2Epreprocessed', 'readmes.csv', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'subject_feats', 'wesad_models_results-win60stride1.csv', 'wesad_models_results-win60stride1_wcm_wcr.csv', 'wesad_readme.pdf']


In [203]:
reduced = df.drop(['label','subject'], axis=1)

In [281]:
reduced.shape

(577, 92)

In [183]:
from sklearn.model_selection import train_test_split

def split_data(df, test_size=0.2, random_state=0):
    # split data into features and labels
    X = df.drop('label', axis=1).values
    y = df['label'].values  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)  
    return [X_train, X_test, y_train, y_test]

the_splits = split_data(df)


In [280]:
np.shape(X_train)

(461, 93)

In [271]:
df.drop(columns=['label'])


,net_acc_C_mean,net_acc_C_std,net_acc_C_min,net_acc_C_max,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,EDA_mean,EDA_std,EDA_min,EDA_max,EDA_SCR_mean,EDA_SCR_std,EDA_SCR_min,EDA_SCR_max,EDA_SCL_mean,EDA_SCL_std,EDA_SCL_min,EDA_SCL_max,BVP_mean,BVP_std,BVP_min,BVP_max,TEMP_mean,TEMP_std,TEMP_min,TEMP_max,ACC_x_mean,ACC_x_std,ACC_x_min,ACC_x_max,ACC_y_mean,ACC_y_std,ACC_y_min,ACC_y_max,ACC_z_mean,ACC_z_std,ACC_z_min,ACC_z_max,ECG_mean,ECG_std,ECG_min,ECG_max,ACC_x_C_mean,ACC_x_C_std,ACC_x_C_min,ACC_x_C_max,ACC_y_C_mean,ACC_y_C_std,ACC_y_C_min,ACC_y_C_max,ACC_z_C_mean,ACC_z_C_std,ACC_z_C_min,ACC_z_C_max,Resp_C_mean,Resp_C_std,Resp_C_min,Resp_C_max,BVP_peak_freq,net_acc_abs_integral,ACC_x_abs_integral,ACC_y_abs_integral,ACC_z_abs_integral,net_acc_C_abs_integral,ACC_x_C_abs_integral,ACC_y_C_abs_integral,ACC_z_C_abs_integral,ACC_x_peak_freq,ACC_y_peak_freq,ACC_z_peak_freq,ACC_x_C_peak_freq,ACC_y_C_peak_freq,ACC_z_C_peak_freq,ECG_bpm,ECG_ibi,ECG_sdnn,ECG_sdsd,ECG_rmssd,ECG_pnn20,ECG_pnn50,EDA_slope,EDA_drange,Resp_C_rate,Resp_C_Inhal_mean,Resp_C_Inhal_std,Resp_C_Exhal_mean,Resp_C_Exhal_std,Resp_C_I/E,TEMP_drange,TEMP_slope,subject
0,0.025961,0.013811,0.000000,0.087383,0.025961,0.013811,0.000000,0.087383,1.303625,0.151736,1.014138,1.678399,0.001666,0.220098,-0.570261,0.837062,2.768776,0.391633,2.357121,3.552635,-0.505818,113.329946,-392.28,554.77,35.807285,0.024986,35.75,35.87,0.023431,0.017769,-0.037843,0.087383,0.000016,1.222594e-05,-0.000026,0.000060,0.000016,1.222594e-05,-0.000026,0.000060,0.002028,0.138996,-0.435883,0.979294,0.000531,0.000086,0.000371,0.000788,3.651495e-07,5.919340e-08,2.551719e-07,5.422521e-07,3.651495e-07,5.919340e-08,2.551719e-07,5.422521e-07,0.092862,2.890267,-8.805847,6.742859,0.081425,45.899363,45.899341,0.031581,0.031581,45.899363,20.529582,0.014125,0.014125,0.004525,0.004525,0.004525,0.000207,0.000207,0.000207,83.340850,719.935065,78.268575,41.148065,54.956383,0.600000,0.200000,-0.001824,1.655001,0.255831,1.911429,0.622539,1.839898,0.741267,1.113083,1.003357,-0.000253,2
1,0.027640,0.010597,0.002752,0.054356,0.027640,0.010597,0.002752,0.054356,0.892549,0.142093,0.693996,1.190967,0.000252,0.038106,-0.115724,0.156043,1.521281,0.428087,0.940590,2.352825,0.523661,41.901619,-289.26,209.89,35.706833,0.024641,35.66,35.75,0.027640,0.010597,0.002752,0.054356,0.000019,7.290999e-06,0.000002,0.000037,0.000019,7.290999e-06,0.000002,0.000037,0.000558,0.153711,-0.302902,0.862701,0.000426,0.000004,0.000400,0.000443,2.933972e-07,2.884456e-09,2.749607e-07,3.049753e-07,2.933972e-07,2.884456e-09,2.749607e-07,3.049753e-07,0.037853,1.596604,-2.914429,3.730774,0.147017,48.895148,48.895124,0.033642,0.033642,48.895148,16.495495,0.011350,0.011350,0.004522,0.004522,0.004522,0.001034,0.001034,0.001034,70.332124,853.095238,76.485367,37.320220,53.633471,0.618182,0.309091,-0.002173,1.716100,0.303811,1.414436,0.189863,1.487778,0.439986,1.003521,1.002524,-0.000161,2
2,0.028389,0.006937,0.000000,0.066053,0.028389,0.006937,0.000000,0.066053,0.598712,0.066870,0.486494,0.718106,0.000348,0.023755,-0.054433,0.092293,0.628467,0.201537,0.316546,0.938482,0.095230,58.836498,-367.11,363.29,35.775430,0.037082,35.71,35.84,0.028378,0.006985,-0.002752,0.066053,0.000020,4.805734e-06,-0.000002,0.000045,0.000020,4.805734e-06,-0.000002,0.000045,0.001483,0.184627,-1.499405,1.415726,0.000427,0.000004,0.000407,0.000444,2.939474e-07,2.705818e-09,2.799789e-07,3.058275e-07,2.939474e-07,2.705818e-09,2.799789e-07,3.058275e-07,0.058251,1.591127,-3.242493,3.450012,0.088210,50.192129,50.192105,0.034535,0.034535,50.192129,16.526430,0.011371,0.011371,0.040724,0.040724,0.040724,0.000207,0.000207,0.000207,72.739014,824.866828,54.526114,29.440553,47.870672,0.673077,0.250000,-0.001035,1.476084,0.292811,1.400794,0.345404,1.431053,0.437193,0.927337,1.003640,0.000535,2
3,0.033268,0.007670,0.000000,0.074998,0.033268,0.007670,0.000000,0.074998,0.504760,0.018053,0.470152,0.568963,0.000345,0.044965,-0.100666,0.183223,0.343030,0.020669,0.308810,0.388085,-0.179239,84.224843,-670.

In [275]:
df

xx = df.drop(columns=['label'])
yy = df['label']
#xx.drop('label', axis=1, inplace=True)

In [276]:
display(xx)
display(yy)


,net_acc_C_mean,net_acc_C_std,net_acc_C_min,net_acc_C_max,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,EDA_mean,EDA_std,EDA_min,EDA_max,EDA_SCR_mean,EDA_SCR_std,EDA_SCR_min,EDA_SCR_max,EDA_SCL_mean,EDA_SCL_std,EDA_SCL_min,EDA_SCL_max,BVP_mean,BVP_std,BVP_min,BVP_max,TEMP_mean,TEMP_std,TEMP_min,TEMP_max,ACC_x_mean,ACC_x_std,ACC_x_min,ACC_x_max,ACC_y_mean,ACC_y_std,ACC_y_min,ACC_y_max,ACC_z_mean,ACC_z_std,ACC_z_min,ACC_z_max,ECG_mean,ECG_std,ECG_min,ECG_max,ACC_x_C_mean,ACC_x_C_std,ACC_x_C_min,ACC_x_C_max,ACC_y_C_mean,ACC_y_C_std,ACC_y_C_min,ACC_y_C_max,ACC_z_C_mean,ACC_z_C_std,ACC_z_C_min,ACC_z_C_max,Resp_C_mean,Resp_C_std,Resp_C_min,Resp_C_max,BVP_peak_freq,net_acc_abs_integral,ACC_x_abs_integral,ACC_y_abs_integral,ACC_z_abs_integral,net_acc_C_abs_integral,ACC_x_C_abs_integral,ACC_y_C_abs_integral,ACC_z_C_abs_integral,ACC_x_peak_freq,ACC_y_peak_freq,ACC_z_peak_freq,ACC_x_C_peak_freq,ACC_y_C_peak_freq,ACC_z_C_peak_freq,ECG_bpm,ECG_ibi,ECG_sdnn,ECG_sdsd,ECG_rmssd,ECG_pnn20,ECG_pnn50,EDA_slope,EDA_drange,Resp_C_rate,Resp_C_Inhal_mean,Resp_C_Inhal_std,Resp_C_Exhal_mean,Resp_C_Exhal_std,Resp_C_I/E,TEMP_drange,TEMP_slope,subject
0,0.025961,0.013811,0.000000,0.087383,0.025961,0.013811,0.000000,0.087383,1.303625,0.151736,1.014138,1.678399,0.001666,0.220098,-0.570261,0.837062,2.768776,0.391633,2.357121,3.552635,-0.505818,113.329946,-392.28,554.77,35.807285,0.024986,35.75,35.87,0.023431,0.017769,-0.037843,0.087383,0.000016,1.222594e-05,-0.000026,0.000060,0.000016,1.222594e-05,-0.000026,0.000060,0.002028,0.138996,-0.435883,0.979294,0.000531,0.000086,0.000371,0.000788,3.651495e-07,5.919340e-08,2.551719e-07,5.422521e-07,3.651495e-07,5.919340e-08,2.551719e-07,5.422521e-07,0.092862,2.890267,-8.805847,6.742859,0.081425,45.899363,45.899341,0.031581,0.031581,45.899363,20.529582,0.014125,0.014125,0.004525,0.004525,0.004525,0.000207,0.000207,0.000207,83.340850,719.935065,78.268575,41.148065,54.956383,0.600000,0.200000,-0.001824,1.655001,0.255831,1.911429,0.622539,1.839898,0.741267,1.113083,1.003357,-0.000253,2
1,0.027640,0.010597,0.002752,0.054356,0.027640,0.010597,0.002752,0.054356,0.892549,0.142093,0.693996,1.190967,0.000252,0.038106,-0.115724,0.156043,1.521281,0.428087,0.940590,2.352825,0.523661,41.901619,-289.26,209.89,35.706833,0.024641,35.66,35.75,0.027640,0.010597,0.002752,0.054356,0.000019,7.290999e-06,0.000002,0.000037,0.000019,7.290999e-06,0.000002,0.000037,0.000558,0.153711,-0.302902,0.862701,0.000426,0.000004,0.000400,0.000443,2.933972e-07,2.884456e-09,2.749607e-07,3.049753e-07,2.933972e-07,2.884456e-09,2.749607e-07,3.049753e-07,0.037853,1.596604,-2.914429,3.730774,0.147017,48.895148,48.895124,0.033642,0.033642,48.895148,16.495495,0.011350,0.011350,0.004522,0.004522,0.004522,0.001034,0.001034,0.001034,70.332124,853.095238,76.485367,37.320220,53.633471,0.618182,0.309091,-0.002173,1.716100,0.303811,1.414436,0.189863,1.487778,0.439986,1.003521,1.002524,-0.000161,2
2,0.028389,0.006937,0.000000,0.066053,0.028389,0.006937,0.000000,0.066053,0.598712,0.066870,0.486494,0.718106,0.000348,0.023755,-0.054433,0.092293,0.628467,0.201537,0.316546,0.938482,0.095230,58.836498,-367.11,363.29,35.775430,0.037082,35.71,35.84,0.028378,0.006985,-0.002752,0.066053,0.000020,4.805734e-06,-0.000002,0.000045,0.000020,4.805734e-06,-0.000002,0.000045,0.001483,0.184627,-1.499405,1.415726,0.000427,0.000004,0.000407,0.000444,2.939474e-07,2.705818e-09,2.799789e-07,3.058275e-07,2.939474e-07,2.705818e-09,2.799789e-07,3.058275e-07,0.058251,1.591127,-3.242493,3.450012,0.088210,50.192129,50.192105,0.034535,0.034535,50.192129,16.526430,0.011371,0.011371,0.040724,0.040724,0.040724,0.000207,0.000207,0.000207,72.739014,824.866828,54.526114,29.440553,47.870672,0.673077,0.250000,-0.001035,1.476084,0.292811,1.400794,0.345404,1.431053,0.437193,0.927337,1.003640,0.000535,2
3,0.033268,0.007670,0.000000,0.074998,0.033268,0.007670,0.000000,0.074998,0.504760,0.018053,0.470152,0.568963,0.000345,0.044965,-0.100666,0.183223,0.343030,0.020669,0.308810,0.388085,-0.179239,84.224843,-670.

0      1
1      1
2      1
3      1
4      1
      ..
572    0
573    0
574    0
575    0
576    0
Name: label, Length: 577, dtype: int64

In [205]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

#display(the_splits[0])
X_train, X_test, y_train, y_test = the_splits
clf = SVC(kernel='linear', C=1, random_state=0)
clf.fit(X_train, y_train)
y_out = clf.predict(X_test)    

svm_accuracy  = accuracy_score(y_test, y_out )

In [261]:
print(X_train, X_test, y_train, y_test)

[[ 3.26681431e-02  6.99495783e-03  2.75221410e-03 ...  1.00177778e+00
  -1.12955407e-04  8.00000000e+00]
 [ 4.36026679e-03  4.57285453e-03  0.00000000e+00 ...  1.00259488e+00
   1.13044348e-04  3.00000000e+00]
 [ 2.91426471e-02  2.79981310e-03  2.13296593e-02 ...  1.00259656e+00
  -1.66120048e-04  1.60000000e+01]
 ...
 [ 3.68762442e-02  6.84362139e-03  0.00000000e+00 ...  1.00203844e+00
   1.29542953e-04  1.10000000e+01]
 [ 4.17490486e-02  1.12975660e-03  3.78429438e-02 ...  1.00708592e+00
  -2.61509556e-04  7.00000000e+00]
 [ 4.03143356e-02  6.37099378e-03  2.06416057e-03 ...  1.00461538e+00
  -5.89875024e-04  1.70000000e+01]] [[ 2.74342382e-02  1.29903815e-02  0.00000000e+00 ...  1.00412858e+00
   4.80082715e-04  6.00000000e+00]
 [ 3.83909752e-02  1.15998846e-02  8.25664229e-03 ...  1.00186162e+00
  -1.66675931e-04  1.40000000e+01]
 [ 3.92529087e-02  7.38493436e-04  2.68340875e-02 ...  1.00179695e+00
  -3.44425050e-05  8.00000000e+00]
 ...
 [ 3.88822197e-02  1.38367626e-03  2.3393819

In [211]:
old_wesad = pd.read_csv('../data/WESAD/subject_feats/oct5_feats4.csv', index_col=0)
new_wesad = pd.read_csv('../data/WESAD/subject_feats/WESADfeatures-win60stride1.csv', index_col=0)

In [213]:
gn_wesad = pd.read_csv('../data/GN-WESAD/n_0/snr_0.0001/subject_feats/2023-11-13_feats2.csv', index_col=0)

In [220]:
for snr in snrs:
    for n_i in range(n_samples):
        tgt = f'../data/GN-WESAD/n_{n_i}/snr_{snr}{subject_feature_path}/{gn_wesad_day}_feats2.csv'
        print(f'df {snr} n {n_i} shape is {(pd.read_csv(tgt, index_col=0)).shape}')

df 0.0001 n 0 shape is (3355, 96)
df 0.0001 n 1 shape is (3355, 96)
df 0.0001 n 2 shape is (3355, 96)
df 0.0001 n 3 shape is (3355, 96)
df 0.0001 n 4 shape is (3355, 96)
df 0.0001 n 5 shape is (3355, 96)
df 0.0001 n 6 shape is (3355, 96)
df 0.0001 n 7 shape is (3355, 96)
df 0.0001 n 8 shape is (3355, 96)
df 0.0001 n 9 shape is (3355, 96)
df 0.001 n 0 shape is (3355, 96)
df 0.001 n 1 shape is (3355, 96)
df 0.001 n 2 shape is (3355, 96)
df 0.001 n 3 shape is (3355, 96)
df 0.001 n 4 shape is (3355, 96)
df 0.001 n 5 shape is (3355, 96)
df 0.001 n 6 shape is (3355, 96)
df 0.001 n 7 shape is (3355, 96)
df 0.001 n 8 shape is (3355, 96)
df 0.001 n 9 shape is (3355, 96)
df 0.01 n 0 shape is (3355, 96)
df 0.01 n 1 shape is (3355, 96)
df 0.01 n 2 shape is (3355, 96)
df 0.01 n 3 shape is (3355, 96)
df 0.01 n 4 shape is (3355, 96)
df 0.01 n 5 shape is (3355, 96)
df 0.01 n 6 shape is (3355, 96)
df 0.01 n 7 shape is (3355, 96)
df 0.01 n 8 shape is (3355, 96)
df 0.01 n 9 shape is (3355, 96)
df 0.05 n 

In [259]:
print('features used for fb_models\n', *old_wesad.columns, sep=',\n')

features used for fb_models
,
net_acc_C_mean,
net_acc_C_std,
net_acc_C_min,
net_acc_C_max,
net_acc_mean,
net_acc_std,
net_acc_min,
net_acc_max,
EDA_mean,
EDA_std,
EDA_min,
EDA_max,
EDA_SCR_mean,
EDA_SCR_std,
EDA_SCR_min,
EDA_SCR_max,
EDA_SCL_mean,
EDA_SCL_std,
EDA_SCL_min,
EDA_SCL_max,
BVP_mean,
BVP_std,
BVP_min,
BVP_max,
TEMP_mean,
TEMP_std,
TEMP_min,
TEMP_max,
ACC_x_mean,
ACC_x_std,
ACC_x_min,
ACC_x_max,
ACC_y_mean,
ACC_y_std,
ACC_y_min,
ACC_y_max,
ACC_z_mean,
ACC_z_std,
ACC_z_min,
ACC_z_max,
ECG_mean,
ECG_std,
ECG_min,
ECG_max,
EDA_C_mean,
EDA_C_std,
EDA_C_min,
EDA_C_max,
EDA_SCR_C_mean,
EDA_SCR_C_std,
EDA_SCR_C_min,
EDA_SCR_C_max,
EDA_SCL_C_mean,
EDA_SCL_C_std,
EDA_SCL_C_min,
EDA_SCL_C_max,
ACC_x_C_mean,
ACC_x_C_std,
ACC_x_C_min,
ACC_x_C_max,
ACC_y_C_mean,
ACC_y_C_std,
ACC_y_C_min,
ACC_y_C_max,
ACC_z_C_mean,
ACC_z_C_std,
ACC_z_C_min,
ACC_z_C_max,
EMG_C_mean,
EMG_C_std,
EMG_C_min,
EMG_C_max,
Resp_C_mean,
Resp_C_std,
Resp_C_min,
Resp_C_max,
Temp_C_mean,
Temp_C_std,
Temp_C_min,
Temp_C

In [258]:
print('features used for fb_models\n', *new_wesad.columns, sep='\n')

features used for fb_models

EDA_mean
EDA_std
EDA_min
EDA_max
EDA_SCR_mean
EDA_SCR_std
EDA_SCR_min
EDA_SCR_max
EDA_SCL_mean
EDA_SCL_std
EDA_SCL_min
EDA_SCL_max
BVP_mean
BVP_std
BVP_min
BVP_max
TEMP_mean
TEMP_std
TEMP_min
TEMP_max
ACC_x_mean
ACC_x_std
ACC_x_min
ACC_x_max
ACC_y_mean
ACC_y_std
ACC_y_min
ACC_y_max
ACC_z_mean
ACC_z_std
ACC_z_min
ACC_z_max
ECG_mean
ECG_std
ECG_min
ECG_max
ACC_x_C_mean
ACC_x_C_std
ACC_x_C_min
ACC_x_C_max
ACC_y_C_mean
ACC_y_C_std
ACC_y_C_min
ACC_y_C_max
ACC_z_C_mean
ACC_z_C_std
ACC_z_C_min
ACC_z_C_max
Resp_C_mean
Resp_C_std
Resp_C_min
Resp_C_max
net_acc_w_mean
net_acc_w_std
net_acc_w_min
net_acc_w_max
net_acc_C_mean
net_acc_C_std
net_acc_C_min
net_acc_C_max
BVP_peak_freq
net_acc_abs_integral
ACC_x_abs_integral
ACC_y_abs_integral
ACC_z_abs_integral
net_acc_C_abs_integral
ACC_x_C_abs_integral
ACC_y_C_abs_integral
ACC_z_C_abs_integral
ACC_x_peak_freq
ACC_y_peak_freq
ACC_z_peak_freq
ACC_x_C_peak_freq
ACC_y_C_peak_freq
ACC_z_C_peak_freq
ECG_bpm
ECG_ibi
ECG_sdnn
ECG

In [26]:
import os
from scipy import signal
from time import time
import concurrent.futures
import itertools
import pandas as pd
import logging
import pickle


subject_ids = 17#[2]#, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
snrs = [ 0.01,0.1]#,0.6 0.05,  0.3,0.1, 0.15, 0.2,0.0001, 0.001, 0.4, 0.5, 0.6] # this is what we ran #0.00001,
n_samples = 0 #10
n_i = n_samples
snr1 = 0.01
snr2 = 0.6
subject_id = 17

sesh_id =[n_i,snr,subject_id]
rt_pth = '/mnt/d/Users/alkurdi/data/GN-WESAD'
start_time = time()
onedrive = '/mnt/d/Users/alkurdi/OneDrive - University of Illinois - Urbana/data/GN-WESAD'

print(os.path.isfile(rt_pth + '/n_'+str(0)+'/snr_'+str(snr1)+ '/S'+str(subject_id)+'/S'+str(subject_id)+'.pkl'))
sesh_path = rt_pth + '/n_'+str(0)+'/snr_'+str(snr1)+ '/S'+str(subject_id)+'/S'+str(subject_id)+'.pkl'
drive_path = onedrive +  '/n_'+str(0)+'/snr_'+str(snr1)+ '/S'+str(subject_id)+'/S'+str(subject_id)+'.pkl'
with open( drive_path, 'rb') as dest:
        ws1_df = pickle.load(dest) 
        #f1=pickle.load(dest)
ws1_df = pd.read_pickle(drive_path)


print(os.listdir(rt_pth + '/n_'+str(0)+'/snr_'+str(snr)+'/S'+str(subject_id)))
#ws1_df = pd.read_pickle(rt_pth + '/n_'+str(0)+'/snr_'+str(snr)+'/S'+str(subject_id)+ '/S'+str(subject_id)+'.pkl')
ws2_df = pd.read_pickle(rt_pth + '/n_'+str(0)+'/snr_'+str(snr2)+'/S'+str(subject_id)+ '/S'+str(subject_id)+'.pkl')

True
['S17.pkl']


In [27]:
ws2_df

{'ACC': array([[ 1409.56589042,  -548.52757719, -3250.84141627],
        [-3669.42211005,   807.27244996,  -757.68229018],
        [ 3049.12852217,   851.41953295, -4146.11284031],
        ...,
        [  233.83393987,   279.05551055,  1158.60451185],
        [-1013.39966173, -1256.77039028,   331.28967768],
        [ 1193.2423905 ,   757.43597076,    83.66353706]]),
 'BVP': array([[  778.14422933],
        [ 2931.20979969],
        [-2846.05497   ],
        ...,
        [  743.99087844],
        [ 2510.29599747],
        [ 2593.27358359]]),
 'EDA': array([[ 1.87297697],
        [ 1.32940784],
        [ 0.76226615],
        ...,
        [ 4.95031892],
        [ 0.47413385],
        [-1.33442696]]),
 'TEMP': array([[  -20.93100476],
        [ 1410.57728896],
        [ 1564.16425681],
        ...,
        [  177.68651182],
        [-3298.94204403],
        [-1602.55805191]]),
 'ACC_C': array([[ 0.91615058,  0.10144717, -1.89996916],
        [-0.01560157, -0.28004619, -1.6985831 ],
      